## Floating Storage time series split by storage location

In this notebook, we will show you how to query floating storage time series alone, as well as floating storage time series split by storage location efficiently.


## Part 1. Query for floating storage time series without split

### 1. Import Libraries and load products

In [9]:
from datetime import datetime
import vortexasdk as v
import pandas as pd
import plotly.express as px
import re
import numpy as np
from dateutil.relativedelta import relativedelta

In [3]:
gasoline = v.Products().search('Gasoline/Blending Components',exact_term_match=True).to_df().loc[0,'id']

2024-09-17 14:25:08,120 vortexasdk.client — WARNING — You are using vortexasdk version 0.73.0, however version 0.73.1 is available.
You should consider upgrading via the 'pip install vortexasdk --upgrade' command.


### 2. Extract data via CargoTimeSeries endpoint

In [7]:
timeseries_df = v.CargoTimeSeries().search(
    filter_activity='storing_state',
    filter_time_min = datetime(2024,3,1),
    filter_time_max = datetime.today(),
    filter_products= gasoline,
    timeseries_unit='b',
    timeseries_frequency='day').to_df()

In [10]:
px.line(timeseries_df, x='key', y='value', title='Gasoline/Blending Components in storage', labels={'value':'Volume (b)'})

## Part 2. Query Floating Storage data split by storage locations

### 1. Helper function

In [5]:
# Load all vessels
vessels = v.Vessels().search().to_df().set_index('id')
print('Loaded {:,} vessels from DB'.format(vessels.shape[0]))
vessels.head()

Loaded 14,997 vessels from DB


,name,imo,vessel_class
id,,,
62f3f3c1f5a663d621fe6cf9537c7d936b547497932f5d713908bc668a9b0412,\tATHINEA,9291248.0,oil_lr2
e6b259c04da30a57db353665e7e61f67a0a3222b96c45738087f6c9ccba22440,\tBORA,9276004.0,oil_mr2
f351708121bce4d357ac5fad967cb1bf7fe5072773f05a78cce51e593e9807dd,0051-04,,oil_coastal
1761da4fb069cd6ce153b6ad1c48e15cdb994eb386e4aafbe8f1bbde993cbaef,058,,oil_coastal
c817b5994efe14621949533d6777b22ce11db1c6bf9e48be826332468141e4e2,1011,,oil_coastal


In [6]:
def extract_vortexa_fs_data(product,start_date,end_date):
    floating_storage = v.CargoMovements().search(
        filter_activity='storing_state',
        filter_time_min = start_date,
        filter_time_max = end_date,
        filter_products = product).to_df(columns = 'all')
    return floating_storage

def find_max_num_storage_events(fs_cols: list) -> int:
    """
    Identifies the maximum number of storage events associated with a single cargo movement
    I.e. The function finds the maximum index in 'events.cargo_storage_event.{ind}.vessel_id' columns
    
    Parameters
    ---------
    fs_cols : list
        A list containing all the relevant columns to floating storage events
        
    Returns
    -------
    max_num : int
        The maximum number of identified storage events per cargo movement
    """
    ind = [int(re.search("events.cargo_storage_event.(\d+)", c).group(1)) for c in fs_cols]
    max_num = max(ind)+1
    return max_num


def replicate_fs_columns(base_fs_cols: list, max_num_fs: int) -> list:
    """
    Helper function to replicate a subset of columns related to FS events
    
    Parameters
    ---------
    base_fs_cols : list
        A list of FS related columns associated with a single FS event / index
        
    max_num_fs: int
        The maximum number of storage events associated with a single cargo movement
        
    Returns
    -------
    final_cols : list
        The final column list
        
    Example: If base_fs_cols = ['event.0.vessel_id', 'event.0.country'] and max_num_fs = 3, the funtion
    will return ['event.0.vessel_id', 'event.0.country', 'event.1.vessel_id', 'event.1.country', 
                 'event.2.vessel_id', 'event.2.country']
    """
    final_cols = []
    for i in range(0, max_num_fs):
        l=[]
        for j in base_fs_cols:
            l.append(j.replace('.0.', f'.{i}.'))
        final_cols.append(l)
    return final_cols


def unnest_fs(df: pd.DataFrame) -> pd.DataFrame:
    """  
    Unnests the FS events and returns a DataFrame where each row corresponds to a single FS event.
    
     Parameters
    ---------
    df : pd.DataFrame
        DataFrame with Floating Storage events as returned from API (with all columns returned)
    
    vessels : pd.DataFrame
        DataFrame with vessel info. Will be used to include vessel_class info for the vessels that took part
        in floating storage events
        
    Returns
    -------
    dc : pd.DataFrame
        DataFrame with one floating storage event per row
    """
    
    # Define default columns to include
    default_cols = ['quantity',
                    'product.group.id',
                    'product.group.label',
                    'product.group_product.id',
                    'product.group_product.label',
                    'product.category.id',
                    'product.category.label',
                    'product.grade.id',
                    'product.grade.label',
                    'cargo_movement_id',
                    'vessels.0.name',
                    'vessels.0.imo',
                    'vessels.0.vessel_class',
                    'vessels.0.dwt',
#                     'vessels.1.name',
#                     'vessels.1.imo',
#                     'vessels.1.vessel_class',
#                     'vessels.1.dwt',
#                     'vessels.2.name',
#                     'vessels.2.imo',
#                     'vessels.2.vessel_class',
#                     'vessels.2.dwt',
                    'events.cargo_port_load_event.0.location.country.label',
                    'events.cargo_port_unload_event.0.location.country.label']
    
    # Find all FS related cols
    storage_cols_all = [c for c in df.columns if 'storage' in c]
    
    # Find max number of floating storage events per cargo movement
    max_num_fs_events = find_max_num_storage_events(storage_cols_all)
    print('Max num fs events',max_num_fs_events)
    # Define subset of FS columns
    storage_base_cols = ['events.cargo_storage_event.0.vessel_id', 
                         'events.cargo_storage_event.0.start_timestamp',
                         'events.cargo_storage_event.0.end_timestamp',
                        #  'events.cargo_storage_event.0.location.country.id',
                        #  'events.cargo_storage_event.0.location.country.label',
                        #  'events.cargo_storage_event.0.location.region.id',
                        #  'events.cargo_storage_event.0.location.region.label',
                         'events.cargo_storage_event.0.location.shipping_region_v2.id',
                         'events.cargo_storage_event.0.location.shipping_region_v2.label']
#                          'events.cargo_storage_event.0.location.trading_region.id',
#                          'events.cargo_storage_event.0.location.trading_region.label',
#                          'events.cargo_storage_event.0.location.trading_subregion.id',
#                          'events.cargo_storage_event.0.location.trading_subregion.label']
                             
    # Replicate FS base cols
    storage_cols = replicate_fs_columns(storage_base_cols, max_num_fs_events)
    
    # Unnest the FS events
    final_cols = [c + default_cols for c in storage_cols]
    
    # Create one DataFrame per 'cluster' and concatenate
    df_list = []
    for group in final_cols:
        df_list.append(df[group])
        
    # Specify final column names
    fs_final_cols = ['fs_vessel_id',
                     'fs_start_timestamp',
                     'fs_end_timestamp',
                    #  'fs_country_id',
                    #  'fs_country_name',
                    #  'fs_region_id',
                    #  'fs_region_name',
                     'fs_shipping_region_id',
                     'fs_shipping_region_name',
                    #  'fs_trading_region_id',
                    #  'fs_trading_region_name',
                    #  'fs_trading_subregion_id',
                    #  'fs_trading_subregion_name',
                     'quantity',
                     'product_id',
                     'product_name',
                     'group_product_id',
                     'group_product_name',
                     'product_category_id',
                     'product_category_name',
                     'grade_id',
                     'grade_name',
                     'cargo_movement_id',
                     'vessels.0.name',
                     'vessels.0.imo',
                     'vessels.0.vessel_class',
                     'vessels.0.dwt',
#                      'vessels.1.name',
#                      'vessels.1.imo',
#                      'vessels.1.vessel_class',
#                      'vessels.1.dwt',
#                      'vessels.2.name',
#                      'vessels.2.imo',
#                      'vessels.2.vessel_class',
#                      'vessels.2.dwt',
                    'events.cargo_port_load_event.0.location.country.label',
                    'events.cargo_port_unload_event.0.location.country.label']
                     
    # Concatenate the elements of the previous list
    # Keep only DataFrame values (otherwise we wouldn't be able to concat since DFs have different column names)
    df_vals = [df_group.values for df_group in df_list]
    df_final = pd.DataFrame(np.concatenate(df_vals, axis=0))
    df_final.columns = fs_final_cols
    df_final = df_final[df_final['fs_vessel_id'].notnull()]
    
    # Covert to datetime columns
    datetime_cols = ['fs_start_timestamp', 'fs_end_timestamp']
    for c in datetime_cols:
        df_final[c] = pd.to_datetime(df_final[c]).dt.tz_localize(None)
        
    # Bring additional vessel info
    df_final = df_final.join(vessels, on = 'fs_vessel_id')
    df_final = df_final.rename(columns = {'name': 'fs_vessel_name',
                                          'imo': 'fs_vessel_imo',
                                          'vessel_class': 'fs_vessel_class'})
    
    # Reorder columns
    df_final = df_final[df_final.columns[-3:].tolist() + df_final.columns[:-3].tolist()]
    
    return df_final

In [7]:
def fs_time_series(df,start_date,end_date):
    # Initialize time series
    idx = pd.date_range(start_date, end_date, freq="D").tz_localize("UTC")
    ts = pd.DataFrame(idx,columns = ['Date'])
    ts['daily_fs_quantity'] = 0
    ts['cargo_fs_count'] = 0

    # Fill end date with latest date
    df['fs_end_timestamp'] = df['fs_end_timestamp'].fillna(pd.to_datetime('today'))
    
    # Convert timestamp to utc format
    df['fs_end_timestamp'] = pd.to_datetime(df['fs_end_timestamp'],utc=  True)
    df['fs_start_timestamp'] = pd.to_datetime(df['fs_start_timestamp'],utc=  True)
    
    # Loop through each day and aggregate the quantity
    for i,rows in ts.iterrows():
        date = rows['Date']
        for j,rows_df in df.iterrows():
            if (date +relativedelta(days=1) > rows_df['fs_start_timestamp']) & (date <= rows_df['fs_end_timestamp']):
                ts.loc[i,'daily_fs_quantity'] += rows_df['quantity']
                ts.loc[i,'cargo_fs_count'] += 1
#                 ts.loc[i,'list_cm'].append(rows_df['cargo_movement_id'])
    return ts

# Group by storage shipping region and product - Feel free to change it to storage location country, storage location region etc.
def groupby_country_fs_time_series(df,START_DATE,END_DATE):
    df2 = df.groupby(['fs_shipping_region_name','product_name'])\
    .apply(fs_time_series, start_date = START_DATE,end_date = END_DATE).reset_index()
    return df2

def construct_time_series(product,start_date,end_date):
    fs_data = extract_vortexa_fs_data(product,start_date,end_date)
    fs_data = unnest_fs(fs_data)
    print('Found {:,} floating storage events'.format(fs_data.shape[0]))
    fs_data = groupby_country_fs_time_series(fs_data,start_date.strftime("%Y-%m-%d"),end_date.strftime("%Y-%m-%d")).drop(columns = ['level_2'])
    return fs_data

### 2. Select date range and products

In [8]:
# Start editing here
start_date = datetime(2024,3,1)
end_date = datetime.today()
fs_data = construct_time_series(gasoline,start_date,end_date)

Max num fs events 12
Found 22,812 floating storage events


C:\Users\OuiWeinJien\AppData\Local\Temp\ipykernel_28336\1273581414.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = df.groupby(['fs_shipping_region_name','product_name'])\


### 3. Plotting

In [15]:
import plotly.express as px
px.bar(fs_data, x = 'Date', y = 'daily_fs_quantity',color = 'fs_shipping_region_name',barmode = 'stack',title= 'Gasoline Floating Storage (bbls) split by Storage Region').show()